In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_curve, roc_auc_score

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D

In [3]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils  import to_categorical

In [4]:
df=pd.read_csv("Appliance_1.csv")

In [5]:
df.head()

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
0,0,1,0.00000,1.00000,0.0,1.0,0,-0.22789,0,0,0
1,0,1,0.25882,0.96593,0.0,1.0,0,-0.22789,0,0,0
2,1,1,0.50000,0.86603,0.0,1.0,0,-0.22789,0,0,0
3,0,1,0.70711,0.70711,0.0,1.0,0,-0.22789,0,-100,0
4,0,1,0.86603,0.50000,0.0,1.0,100,-0.17234,100,0,1


In [6]:
df.columns

Index(['Occupancy', 'Appliance ID', 'Sin(time)', 'Cos(time)', 'Sin(day)',
       'Cos(day)', 'Power consumption', 'Normalized power', 'P(t)-P(t-1)',
       'P(t)-P(t+1)', 'Class Label'],
      dtype='object')

In [7]:
shuffle(df)

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
15545,1,1,-0.96593,-0.25882,-0.98993,0.14154,100,-0.17234,0,0,0
5510,1,1,-0.50000,-0.86603,-0.71768,-0.69638,0,-0.22789,0,0,0
2501,0,1,0.96593,0.25882,0.97601,-0.21772,100,-0.17234,0,0,4
1246,0,1,-0.50000,0.86603,0.76941,0.63875,100,-0.17234,0,100,4
381,0,1,-0.70711,0.70711,0.25535,0.96685,0,-0.22789,-100,0,2
...,...,...,...,...,...,...,...,...,...,...,...
15577,1,1,0.25882,0.96593,-0.98447,0.17553,100,-0.17234,0,0,0
5602,0,1,0.50000,-0.86603,-0.76389,-0.64535,0,-0.22789,0,0,0
15297,0,1,0.70711,-0.70711,-0.99955,-0.03012,100,-0.17234,0,100,4
1625,0,1,-0.96593,-0.25882,0.91413,0.40543,100,-0.17234,0,0,4


In [8]:
# Split the data into train and test sets
train_size = int(len(df)*0.8)
train, test = df.iloc[:train_size],df.iloc[train_size:]

In [9]:
# Split train and test sets into predictors (X) and outcome (y) variables
x_train = train.drop('Class Label', axis = 1)
y_train = train.loc[:,['Class Label']]
x_test = test.drop('Class Label', axis = 1)
y_test = test.loc[:,['Class Label']]

In [10]:
# Scale the X data 
scaler_x = MinMaxScaler(feature_range = (0,1))
input_scaler = scaler_x.fit(x_train)
train_x_norm = input_scaler.transform(x_train)
test_x_norm = input_scaler.transform(x_test)

In [11]:
# One-hot encode y data
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((14016, 10), (3504, 10), (14016, 5), (3504, 5))

In [13]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [14]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [15]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((14016, 10, 1), (3504, 10, 1), (14016, 5), (3504, 5))

In [16]:
verbose, epochs, batch_size = 1, 50, 32
n_timesteps, n_features = x_train.shape[1], x_train.shape[2]

In [17]:
# Define the model architecture
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(10, 1)))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 16)             64        
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 16)             784       
                                                                 
 conv1d_2 (Conv1D)           (None, 4, 16)             784       
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                1088      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [19]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
#fit the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/50
438/438 [==============================] - 1s 1ms/step - loss: 0.2661 - accuracy: 0.9004
Epoch 2/50
438/438 [==============================] - 1s 1ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 3/50
438/438 [==============================] - 1s 1ms/step - loss: 6.6029e-04 - accuracy: 1.0000
Epoch 4/50
438/438 [==============================] - 0s 1ms/step - loss: 3.7881e-04 - accuracy: 1.0000
Epoch 5/50
438/438 [==============================] - 1s 1ms/step - loss: 1.5596e-04 - accuracy: 1.0000
Epoch 6/50
438/438 [==============================] - 0s 1ms/step - loss: 9.0311e-05 - accuracy: 1.0000
Epoch 7/50
438/438 [==============================] - 0s 1ms/step - loss: 8.5852e-05 - accuracy: 1.0000
Epoch 8/50
438/438 [==============================] - 0s 1ms/step - loss: 4.5602e-05 - accuracy: 1.0000
Epoch 9/50
438/438 [==============================] - 0s 982us/step - loss: 4.9375e-05 - accuracy: 1.0000
Epoch 10/50
438/438 [==============================] - 0s 1ms/step - l

In [21]:
_, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose)
#  return accuracy

110/110 [==============================] - 0s 804us/step - loss: 0.0000e+00 - accuracy: 1.0000


In [22]:
accuracy

1.0

In [23]:
prediction = model.predict(x_test)

110/110 [==============================] - 0s 575us/step


In [24]:
# prediction[1]

In [25]:
y_test

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [26]:
# prediction_NN = np.argmax(prediction[1], axis=-1)

In [27]:
prediction_NN=np.argmax(prediction, axis=-1)

In [28]:
prediction_NN

array([0, 0, 0, ..., 0, 0, 4], dtype=int64)

In [29]:
y_test1 = test.loc[:,['Class Label']]

In [30]:
y_test1=np.array(y_test1)

In [31]:
y_test1

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [4]], dtype=int64)

In [32]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test1, prediction_NN))

Accuracy =  1.0


In [33]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1, prediction_NN)
print(cm)

[[2254    0    0    0]
 [   0  291    0    0]
 [   0    0  290    0]
 [   0    0    0  669]]


In [34]:
from sklearn.metrics import classification_report

# print(class_labels)
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 4']
print(classification_report(y_test1, prediction_NN, target_names=target_names,digits=3))

              precision    recall  f1-score   support

     Class 0      1.000     1.000     1.000      2254
     Class 1      1.000     1.000     1.000       291
     Class 2      1.000     1.000     1.000       290
     Class 4      1.000     1.000     1.000       669

    accuracy                          1.000      3504
   macro avg      1.000     1.000     1.000      3504
weighted avg      1.000     1.000     1.000      3504



In [35]:
# !pip install mlxtend

In [36]:
# from mlxtend.plotting import plot_confusion_matrix

In [37]:
# class_dict = {0: 'Class 0',
#               1: 'Class 1',
#               2: 'Class 2',
#               3: 'Class 3',
#               4: 'Class 4'}

# multiclass = np.array([[9926,   67,   19,   56,    0],
#                        [178,  855,    0,   41,   40],
#                        [268,    0,  848,    0,    0],
#                        [27,   16,    0, 2326,   15],
#                        [0,    1,    0,   36, 6305]])

# fig, ax = plot_confusion_matrix(
#     conf_mat=multiclass,
#     class_names=class_dict.values(),show_absolute=False,show_normed=True,colorbar=True)